In [2]:
pip install virtualenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import math
import pandas as pd 
import numpy as np

In [4]:
# --- 0. COLOR PALETTE DEFINITION ---
c = {
    'white':        '#FFFFFF',
    'black':        '#000000',

    'blue_dark':    '#0F2854',
    'blue_medium':  '#1C4D8D',
    'blue_light':   '#4988C4',
    'blue_pale':    '#BDE8F5',

    'gray_light':   '#E5E5E5',
    'gray_medium':  '#CBCBCB',
    'gray_dark':    '#404040',

    'accent_gold':  '#E2A16F'
}
# --- 1. CORE STYLE FUNCTION ---

def apply_my_style(fig, title, subtitle=""):
    """
    Hàm áp dụng style chuẩn cho toàn bộ biểu đồ.
    """
    fig.update_layout(
        title={
            'text': f"<b style='font-size:26px; color:black;'>{title}</b><br>"
                    f"<span style='font-size:16px; color:#666;'>{subtitle}</span>",
            'x': 0.02,
            'y': 0.90,
        
            'xanchor': 'left',
            'yanchor': 'top',
            'font': {'family': "Montserrat, sans-serif"}
        },

        margin=dict(t=100, l=80, r=40, b=80),

        font=dict(family="Montserrat, sans-serif", size=14),
        paper_bgcolor="white",
        plot_bgcolor="white",
        legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99), # Legend gọn gàng

        xaxis=dict(showgrid=False, linecolor='#2C3E50', linewidth=1, mirror=True),
        yaxis=dict(showgrid=False, linecolor='#2C3E50', linewidth=1, mirror=True),
    )
    
    
    return fig

In [5]:
ACADEMIC_PATH = r'../../data/raw/academic_records.csv'
ADMISSION_PATH = r'../../data/raw/admission.csv'
TEST_PATH = r'../../data/raw/test.csv'

In [6]:
academic_records = pd.read_csv(ACADEMIC_PATH)
admission = pd.read_csv(ADMISSION_PATH)

In [7]:
academic_records.head()

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH
0,f022ed8d1ac1,HK2 2020-2021,2.19,2.02,18,18
1,f022ed8d1ac1,HK1 2022-2023,0.95,2.12,14,7
2,f022ed8d1ac1,HK1 2023-2024,0.81,1.89,29,16
3,f022ed8d1ac1,HK2 2022-2023,1.37,1.93,26,23
4,f022ed8d1ac1,HK2 2023-2024,1.71,1.91,16,13


In [8]:
admission.head(10)

,MA_SO_SV,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN
0,0570116c3448,2018,5,A00,15.86,15.10
1,921cfe1e9ca9,2018,5,A01,15.98,14.98
2,8aeb1516c333,2018,5,A01,17.05,15.18
3,94ff3745a70e,2018,5,A00,18.67,14.78
4,5c5900eb2b53,2018,5,D07,16.67,15.06
5,8d0750fba57a,2018,5,D07,15.64,15.18
6,99cb99df7d82,2018,5,A01,16.62,15.11
7,8fa25c62a675,2018,5,A00,17.08,15.21
8,ec1c686cde60,2018,5,A00,15.37,15.17
9,aa6b745b595b,2021,1,A00,17.74,16.22


In [9]:
admission['MA_SO_SV'].nunique()

30217

In [10]:
academic_records['MA_SO_SV'].nunique()

20381

In [11]:
missing_ids = admission.loc[
    ~admission['MA_SO_SV'].isin(academic_records['MA_SO_SV']),
    'MA_SO_SV'
]

missing_ids


3        94ff3745a70e
5        8d0750fba57a
8        ec1c686cde60
16       3a4219325e9b
23       6b66e9010f75
             ...     
30184    584c61ff6ff1
30189    a360c528b5fb
30196    5fd0320ad6cf
30203    bb8fcc43929c
30210    717f22db15ef
Name: MA_SO_SV, Length: 9836, dtype: str

## NOTE 
- số lượng sinh viên trong bảng admission là 30217 > số lượng sinh viên trong bảng academic_records (20381) -> có 9836 sinh viên ko có dữ liệu 
- giả định 1: sinh viên đã đỗ nhưng ko học (nếu như thời gian nhập học trước năm 2025)
- giả định 2: sinh viên năm nhất mới vào chưa có data (nếu như thời gian nhập học là 2025)
- vì mục đích chính của dự án là dự đoán số tín chỉ sinh viên có thể hoàn thành được -> chỉ chọn những sinh viên có data được lưu 
-> chọn cách merge với parameter ```how = inner``` 

In [12]:
df = pd.merge(academic_records, admission, how='inner', on='MA_SO_SV')

In [13]:
df.head(20)

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN
0,f022ed8d1ac1,HK2 2020-2021,2.19,2.02,18,18,2020,1,A00,23.96,21.72
1,f022ed8d1ac1,HK1 2022-2023,0.95,2.12,14,7,2020,1,A00,23.96,21.72
2,f022ed8d1ac1,HK1 2023-2024,0.81,1.89,29,16,2020,1,A00,23.96,21.72
3,f022ed8d1ac1,HK2 2022-2023,1.37,1.93,26,23,2020,1,A00,23.96,21.72
4,f022ed8d1ac1,HK2 2023-2024,1.71,1.91,16,13,2020,1,A00,23.96,21.72
5,75fc14669ae4,HK2 2022-2023,3.68,3.53,20,20,2018,1,A00,21.03,17.28
6,75fc14669ae4,HK1 2022-2023,3.72,3.48,42,42,2018,1,A00,21.03,17.28
7,75fc14669ae4,HK1 2021-2022,3.19,3.43,36,36,2018,1,A00,21.03,17.28
8,75fc14669ae4,HK2 2021-2022,3.48,3.38,30,30,2018,1,A00,21.03,17.28
9,75fc14669ae4,HK2 2020-2021,3.74,3.48,29,29,2018,1,A00,21.03,17.28


In [14]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 105726 entries, 0 to 105725
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   MA_SO_SV         105726 non-null  str    
 1   HOC_KY           105726 non-null  str    
 2   CPA              105726 non-null  float64
 3   GPA              105726 non-null  float64
 4   TC_DANGKY        105726 non-null  int64  
 5   TC_HOANTHANH     105726 non-null  int64  
 6   NAM_TUYENSINH    105726 non-null  int64  
 7   PTXT             105726 non-null  str    
 8   TOHOP_XT         105726 non-null  str    
 9   DIEM_TRUNGTUYEN  105726 non-null  float64
 10  DIEM_CHUAN       105726 non-null  float64
dtypes: float64(4), int64(3), str(4)
memory usage: 8.9 MB


In [15]:
df.isnull().sum()

MA_SO_SV           0
HOC_KY             0
CPA                0
GPA                0
TC_DANGKY          0
TC_HOANTHANH       0
NAM_TUYENSINH      0
PTXT               0
TOHOP_XT           0
DIEM_TRUNGTUYEN    0
DIEM_CHUAN         0
dtype: int64

In [16]:
df['MA_SO_SV'].nunique()

20381

In [17]:
df['NAM_TUYENSINH'].value_counts()

NAM_TUYENSINH
2020    24975
2021    21381
2019    21292
2018    16737
2022    14216
2023     7125
Name: count, dtype: int64

In [18]:
df['HOC_KY'].value_counts()

HOC_KY
HK1 2023-2024    17099
HK1 2022-2023    15486
HK2 2023-2024    15144
HK2 2022-2023    14504
HK1 2021-2022    12698
HK2 2021-2022    12434
HK1 2020-2021     9186
HK2 2020-2021     9175
Name: count, dtype: int64

### UNIVARIATE ANALYSIS

#### CATEGORICAL FEATURE

##### PTXT VÀ THXT 

In [19]:
# --- 2. XỬ LÝ DỮ LIỆU: TÍNH TỈ LỆ % ---
# Bước 1: Đếm số lượng sinh viên theo từng Năm và Phương thức
df_grouped = admission.groupby(['NAM_TUYENSINH', 'PTXT']).size().reset_index(name='Count')

# Bước 2: Tính tổng sinh viên mỗi năm
df_total = df_grouped.groupby('NAM_TUYENSINH')['Count'].transform('sum')

# Bước 3: Tính phần trăm
df_grouped['Percentage'] = (df_grouped['Count'] / df_total) * 100
df_grouped['Percentage'] = df_grouped['Percentage'].round(1) # Làm tròn 1 số thập phân

# --- 3. CẤU HÌNH MÀU SẮC (Ưu tiên độ tương phản) ---
# Tạo list màu từ palette của bạn để gán vòng lặp
color_sequence = [
    c['blue_dark'],    # Màu chủ đạo 1
    c['accent_gold'],  # Màu nổi bật (Highlight)
    c['blue_light'],   # Màu bổ trợ
    c['gray_dark'],    # Màu trung tính
    c['blue_medium']   # Màu dự phòng
]

# --- 4. VẼ BIỂU ĐỒ STACKED BAR ---
fig3 = go.Figure()

# Lấy danh sách các phương thức duy nhất
unique_methods = df_grouped['PTXT'].unique()

for i, method in enumerate(unique_methods):
    # Lọc data theo phương thức
    df_sub = df_grouped[df_grouped['PTXT'] == method]
    
    fig3.add_trace(go.Bar(
        x=df_sub['NAM_TUYENSINH'],
        y=df_sub['Percentage'],
        name=method,
        marker_color=color_sequence[i % len(color_sequence)], # Lặp lại màu nếu nhiều phương thức
        hovertemplate='%{y}%', # Hiển thị % khi di chuột
        text=df_sub['Percentage'].apply(lambda x: f"{x}%" if x > 5 else ""), # Chỉ hiện số nếu > 5% để đỡ rối
        textposition='auto',
        textfont=dict(color='white' if i != 1 else 'black') # Chữ đen nếu nền là màu Gold
    ))

# --- 5. ÁP DỤNG STYLE VÀ TINH CHỈNH ---
fig3.update_layout(
    barmode='stack', # Chồng lên nhau
    xaxis=dict(
        type='category', # Đảm bảo năm hiển thị dạng chữ (Category), không phải số
        title_text="Năm tuyển sinh"
    ),
    yaxis=dict(
        title_text="Tỉ lệ (%)",
        range=[0, 100] # Cố định trục Y từ 0 đến 100
    )
)

fig3 = apply_my_style(
    fig3,
    title="XU HƯỚNG PHƯƠNG THỨC XÉT TUYỂN (2020-2023)",
    subtitle="Cơ cấu tỉ lệ sinh viên nhập học phân theo hình thức xét tuyển đầu vào"
)

fig3.show()

| TT | Mã  | Tên phương thức xét tuyển |
|----|-----|----------------------------|
| 1  | 100 | Xét kết quả thi tốt nghiệp THPT |
| 2  | 200 | Xét kết quả học tập cấp THPT (học bạ) |
| 3  | 301 | Xét tuyển thẳng theo quy định của Quy chế tuyển sinh của Bộ GD&ĐT (Điều 8) |
| 4  | 302 | Xét kết hợp giữa tuyển thẳng theo Đề án và các phương thức khác |
| 5  | 303 | Xét tuyển thẳng theo Đề án của CSĐT |
| 6  | 401 | Thi đánh giá năng lực, đánh giá tư duy do CSĐT tự tổ chức để xét tuyển |
| 7  | 402 | Sử dụng kết quả thi đánh giá năng lực, đánh giá tư duy do đơn vị khác tổ chức để xét tuyển |
| 8  | 403 | Thi văn hóa do CSĐT tổ chức để xét tuyển |
| 9  | 404 | Sử dụng kết quả thi văn hóa do CSĐT khác tổ chức để xét tuyển |
| 10 | 405 | Kết hợp kết quả thi tốt nghiệp THPT với điểm thi năng khiếu để xét tuyển |
| 11 | 406 | Kết hợp kết quả học tập cấp THPT với điểm thi năng khiếu để xét tuyển |
| 12 | 407 | Kết hợp kết quả thi tốt nghiệp THPT với kết quả học tập cấp THPT để xét tuyển |
| 13 | 408 | Kết hợp chứng chỉ quốc tế với tiêu chí khác để xét tuyển |
| 14 | 409 | Kết hợp kết quả thi tốt nghiệp THPT với chứng chỉ quốc tế để xét tuyển |
| 15 | 410 | Kết hợp kết quả học tập cấp THPT với chứng chỉ quốc tế để xét tuyển |
| 16 | 411 | Xét tuyển thí sinh tốt nghiệp THPT nước ngoài |
| 17 | 412 | Kết hợp phỏng vấn với tiêu chí khác để xét tuyển |
| 18 | 413 | Kết hợp kết quả thi tốt nghiệp THPT với phỏng vấn để xét tuyển |
| 19 | 414 | Kết hợp kết quả học tập cấp THPT với phỏng vấn để xét tuyển |
| 20 | 500 | Sử dụng phương thức khác |


In [ ]:
# --- 2. XỬ LÝ DỮ LIỆU: TÍNH TỈ LỆ % ---
# Bước 1: Đếm số lượng sinh viên theo từng Năm và Phương thức
df_grouped = admission.groupby(['NAM_TUYENSINH', 'THXT']).size().reset_index(name='Count')

# Bước 2: Tính tổng sinh viên mỗi năm
df_total = df_grouped.groupby('NAM_TUYENSINH')['Count'].transform('sum')

# Bước 3: Tính phần trăm
df_grouped['Percentage'] = (df_grouped['Count'] / df_total) * 100
df_grouped['Percentage'] = df_grouped['Percentage'].round(1) # Làm tròn 1 số thập phân

# --- 3. CẤU HÌNH MÀU SẮC (Ưu tiên độ tương phản) ---
# Tạo list màu từ palette của bạn để gán vòng lặp
color_sequence = [
    c['blue_dark'],    # Màu chủ đạo 1
    c['accent_gold'],  # Màu nổi bật (Highlight)
    c['blue_light'],   # Màu bổ trợ
    c['gray_dark'],    # Màu trung tính
    c['blue_medium']   # Màu dự phòng
]

# --- 4. VẼ BIỂU ĐỒ STACKED BAR ---
fig3 = go.Figure()

# Lấy danh sách các phương thức duy nhất
unique_methods = df_grouped['THXT'].unique()

for i, method in enumerate(unique_methods):
    # Lọc data theo phương thức
    df_sub = df_grouped[df_grouped['THXT'] == method]
    
    fig3.add_trace(go.Bar(
        x=df_sub['NAM_TUYENSINH'],
        y=df_sub['Percentage'],
        name=method,
        marker_color=color_sequence[i % len(color_sequence)], # Lặp lại màu nếu nhiều phương thức
        hovertemplate='%{y}%', # Hiển thị % khi di chuột
        text=df_sub['Percentage'].apply(lambda x: f"{x}%" if x > 5 else ""), # Chỉ hiện số nếu > 5% để đỡ rối
        textposition='auto',
        textfont=dict(color='white' if i != 1 else 'black') # Chữ đen nếu nền là màu Gold
    ))

# --- 5. ÁP DỤNG STYLE VÀ TINH CHỈNH ---
fig3.update_layout(
    barmode='stack', # Chồng lên nhau
    xaxis=dict(
        type='category', # Đảm bảo năm hiển thị dạng chữ (Category), không phải số
        title_text="Năm tuyển sinh"
    ),
    yaxis=dict(
        title_text="Tỉ lệ (%)",
        range=[0, 100] # Cố định trục Y từ 0 đến 100
    )
)

fig3 = apply_my_style(
    fig3,
    title="XU HƯỚNG TỔ HỢP XÉT TUYỂN (2020-2023)",
    subtitle="Cơ cấu tỉ lệ sinh viên nhập học phân theo tổ hợp xét tuyển đầu vào"
)

fig3.show()

#### NUMERICAL FEATURE 

In [20]:
fig1 = go.Figure()

# Box cho CPA
fig1.add_trace(go.Box(
    y=df['CPA'],
    name='CPA (Tích lũy)',
    marker_color=c['blue_medium'],
    boxpoints='outliers', # Chỉ hiện điểm ngoại lai để đỡ rối
    line=dict(width=1.5),
    fillcolor=c['blue_pale'] # Fill màu nhạt để thanh thoát
))

# Box cho GPA
fig1.add_trace(go.Box(
    y=df['GPA'],
    name='GPA (Học kỳ)',
    marker_color=c['accent_gold'],
    boxpoints='outliers',
    line=dict(width=1.5),
    fillcolor='#FBEAD8' # Màu gold nhạt pha tay
))

# Áp dụng Style
fig1 = apply_my_style(
    fig1, 
    title="PHÂN PHỐI ĐIỂM SỐ: CPA vs. GPA", 
    subtitle="So sánh mức độ biến động điểm tích lũy và điểm học kỳ toàn trường"
)
fig1.update_yaxes(title_text="Thang điểm 4.0")

fig1.show()



### BIVARIATE ANALYSIS 

#### GPA AND TIME 

In [21]:
import re
import plotly.graph_objects as go

# --- 1. HÀM TẠO KEY SẮP XẾP (LOGIC QUAN TRỌNG NHẤT) ---
def get_sort_key(hocky_str):
    """
    Chuyển đổi chuỗi 'HK2 2020-2021' thành tuple (2020, 2) để sắp xếp.
    Priority 1: Năm học (2020)
    Priority 2: Học kỳ (2)
    """
    # Dùng Regex để bắt số học kỳ và năm bắt đầu
    # Pattern: Tìm chữ "HK" theo sau là số, rồi tìm 4 chữ số (năm)
    match = re.search(r'HK(\d+).*?(\d{4})', str(hocky_str))
    
    if match:
        hocky_num = int(match.group(1)) # Ví dụ: 2
        year_num = int(match.group(2))  # Ví dụ: 2020
        return (year_num, hocky_num)    # Trả về tuple để sort: (Năm, Kỳ)
    
    return (9999, 9999) # Nếu lỗi format thì đẩy xuống cuối

# --- 2. HÀM RÚT GỌN TÊN (NHƯ CŨ) ---
def shorten_semester_name(raw_name):
    # Logic: "HK2 2020-2021" -> "HK02 20-21"
    name = str(raw_name)
    match = re.search(r'HK(\d+).*?(\d{4})-(\d{4})', name)
    if match:
        hk = int(match.group(1))
        y1 = match.group(2)[-2:]
        y2 = match.group(3)[-2:]
        return f"HK{hk:02d} {y1}-{y2}" # Format HK01, HK02...
    return name

# --- 3. XỬ LÝ DỮ LIỆU ---
# Lấy danh sách học kỳ duy nhất
unique_hocky = df['HOC_KY'].unique()

# SẮP XẾP DANH SÁCH THEO LOGIC MỚI
# key=get_sort_key sẽ đảm bảo năm 2020 đứng trước 2021, HK1 đứng trước HK2
sorted_hocky = sorted(unique_hocky, key=get_sort_key)

# --- 4. VẼ BIỂU ĐỒ ---
fig2 = go.Figure()

for hk in sorted_hocky:
    df_hk = df[df['HOC_KY'] == hk]
    
    # Tạo tên hiển thị ngắn gọn
    display_name = shorten_semester_name(hk)
    
    fig2.add_trace(go.Box(
        y=df_hk['GPA'],
        name=display_name, # Tên đã rút gọn (VD: HK01 20-21)
        boxpoints=False, 
        marker_color=c['accent_gold'],
        line=dict(width=1.5),
        fillcolor='#FBEAD8'
    ))

# --- 5. ÁP DỤNG STYLE ---
fig2 = apply_my_style(
    fig2, 
    title="BIẾN ĐỘNG KẾT QUẢ HỌC TẬP QUA CÁC KỲ", 
    subtitle="Phân tích phân phối điểm GPA theo trình tự thời gian (Năm học - Học kỳ)"
)

fig2.update_layout(showlegend=False)

fig2.show()

### NOTE:
- điểm của HK1 20-21 có vẻ thấp nhất -> ảnh hưởng dịch covid 19 khiến chất lượng ko được đảm bảo ?

### NOTE
- có một số sinh viên năm nhập học và năm bắt đầu kì học đầu tiên là khác nhau ?
- có phải là gap year -> tạo biến khoảng cách xem có khai thác được gì không

In [ ]:
import pandas as pd
import numpy as np

def create_semester_features(df_merged):
    """
    Hàm tạo các biến liên quan đến thời gian học, phát hiện nghỉ học.
    Input: df_merged (DataFrame đã gộp admission và academic_records)
    Output: DataFrame đã có thêm các cột tính năng mới.
    """
    # 1. Đảm bảo dữ liệu được sắp xếp đúng để dùng hàm shift()
    # Sắp xếp theo Sinh viên và Học kỳ tăng dần
    df = df_merged.sort_values(by=['MA_SO_SV', 'HOC_KY']).copy()
    
    # 2. Xử lý thông tin Học kỳ và Năm học từ cột HOC_KY
    # Giả định HOC_KY dạng 20231 (Năm 2023, Kỳ 1)
    df['Year_Current'] = df['HOC_KY'] // 10
    df['Sem_Current'] = df['HOC_KY'] % 10
    
    # --- FEATURE 1: KỲ THỨ N THỰC TẾ (Active Semester) ---
    # Đếm xem đây là bản ghi thứ mấy của sinh viên này (không quan tâm thời gian)
    # Ví dụ: Học kỳ đầu tiên xuất hiện là 1, tiếp theo là 2...
    df['Ky_Thu_N_Active'] = df.groupby('MA_SO_SV').cumcount() + 1

    # --- FEATURE 2: KỲ THỨ N THEO LỊCH SỬ (Chronological Semester) ---
    # Tính khoảng cách từ năm nhập học đến kỳ hiện tại.
    # Giả sử 1 năm có 3 học kỳ (1, 2, 3 - tính cả hè để timeline liên tục). 
    # Công thức: (Năm học - Năm nhập học) * 3 + Kỳ học
    # Trừ đi phần dư để kỳ đầu tiên (ví dụ nhập học 2023, kỳ 20231) bắt đầu từ 1.
    
    # Lưu ý: Cần điều chỉnh logic tùy vào kỳ nhập học đầu tiên. 
    # Thường sinh viên nhập học vào kỳ 1.
    df['Ky_Thu_N_Chrono'] = (df['Year_Current'] - df['NAM_TUYENSINH']) * 3 + df['Sem_Current']
    
    # Xử lý trường hợp sinh viên nhập học nhưng dữ liệu bắt đầu bị lệch (optional normalization)
    # Nếu muốn kỳ đầu tiên luôn bắt đầu là 1 (tương đối):
    # min_chrono = df.groupby('MA_SO_SV')['Ky_Thu_N_Chrono'].transform('min')
    # df['Ky_Thu_N_Chrono_Rel'] = df['Ky_Thu_N_Chrono'] - min_chrono + 1

    # --- FEATURE 3: PHÁT HIỆN GAP (NGHỈ HỌC) ---
    # Tính "Kỳ theo lịch" của dòng trước đó
    df['Prev_Chrono'] = df.groupby('MA_SO_SV')['Ky_Thu_N_Chrono'].shift(1)
    
    # Khoảng cách giữa kỳ này và kỳ trước
    # Ví dụ: Kỳ trước là 1, Kỳ này là 2 -> Diff = 1 (Liên tục)
    # Kỳ trước là 1, Kỳ này là 4 (Nghỉ 2 kỳ) -> Diff = 3
    df['Diff_Sem'] = df['Ky_Thu_N_Chrono'] - df['Prev_Chrono']
    
    # Gap_Duration: Số kỳ nghỉ. 
    # Nếu Diff = 1 (liên tục) hoặc NaN (kỳ đầu) -> Gap = 0
    # Nếu Diff > 1 -> Gap = Diff - 1
    df['Gap_Duration'] = df['Diff_Sem'].apply(lambda x: x - 1 if x > 1 else 0).fillna(0).astype(int)
    
    # --- FEATURE 4: CỜ ĐÁNH DẤU QUAY LẠI (Is_Returning) ---
    # True nếu sinh viên vừa trải qua một kỳ nghỉ
    df['Is_Returning'] = df['Gap_Duration'] > 0
    
    # --- FEATURE 5: TỔNG SỐ KỲ ĐÃ NGHỈ TÍCH LŨY ---
    # Cho biết đến thời điểm này, sinh viên đã nghỉ tổng cộng bao nhiêu kỳ
    df['Total_Gap_Accumulated'] = df.groupby('MA_SO_SV')['Gap_Duration'].cumsum()

    # Dọn dẹp các cột tạm nếu không cần thiết
    # df = df.drop(columns=['Year_Current', 'Sem_Current', 'Prev_Chrono', 'Diff_Sem'])
    
    return df

# --- CÁCH SỬ DỤNG ---
# df_processed = create_semester_features(df)
# print(df_processed[['MA_SO_SV', 'HOC_KY', 'Ky_Thu_N_Active', 'Ky_Thu_N_Chrono', 'Gap_Duration', 'Is_Returning']].head(20))